In [1]:
! pip install huggingface_hub langchain accelerate bitsandbytes optimum auto-gptq chromadb sentence-transformers peft tensorflow==2.15
! pip install -U transformers lark
! pip install langchain-community


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import pandas as pd
import os
from langchain.document_loaders import DirectoryLoader
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import re
import json
from langchain.embeddings.sentence_transformer import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
import time
import os
import numpy as np
from langchain.docstore.document import Document
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
from langchain.llms import HuggingFacePipeline
from peft import PeftModel
from transformers import TextStreamer, pipeline
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
import chromadb


# garbage collection automatic
import gc

with open(os.path.join('hugging_face_api.txt')) as f:
    contents = f.read().strip().split('\n')
    os.environ['HFH_API_TOKEN_R'] = contents[0]
    os.environ['HFH_API_TOKEN_W'] = contents[1]
    
from huggingface_hub import login

login(os.environ['HFH_API_TOKEN_R'])
login(os.environ['HFH_API_TOKEN_W'])



def load_vector_db(file_path=os.path.join('metadata.json'), loader=None, **kwargs):
    
    with open(file_path, 'r') as json_file:
        df = json.load(json_file)
    
    documents = []
    for key in df:
        vals = df[key]

        doc = Document(
            page_content= key + "\n" + vals['description'] + "\n" + key,
            metadata={
                "source": vals['link'],
                "Date": vals['pubdate'],
                "title": key
            }
        )
        documents.append(doc)
        
    
    



   

    return documents


def setup_vector_search(data_file_path, tokens_per_chunk=500, chunk_overlap=30, huggingface_embedding_name="BAAI/bge-small-en-v1.5"): 
    embedding_name = huggingface_embedding_name # "BAAI/bge-small-en-v1.5"

    # load the directory and split it into chunks
    print("cwd is: ", os.getcwd())

    print('Loading...')

    documents = load_vector_db(data_file_path, "*.txt", silent_errors=True)

#     text_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=chunk_overlap, model_name=embedding_name, 
#                                                         tokens_per_chunk=tokens_per_chunk)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=chunk_overlap)
    print('Splitting...')
    docs = text_splitter.split_documents(documents)

    embedding_function = HuggingFaceEmbeddings(model_name=embedding_name)
    # load it into Chroma
    print('Embedding...')

    db = Chroma.from_documents(docs, embedding_function)
    # db_retriever = db.as_retriever(search_type="mmr", search_kwargs={'k':6, 'lambda_mult': 0.25})
    return db



device = 'cuda:0'
# model to use
model_id = 'mistralai/Mistral-7B-Instruct-v0.2'

# Useful link to learn more: https://medium.com/@murtuza753/using-llama-2-0-faiss-and-langchain-for-question-answering-on-your-own-data-682241488476


# Tokenizer for turning user input into embeddings
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, device_map="auto", token=os.environ['HFH_API_TOKEN_R'])
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, device_map="auto", token=os.environ['HFH_API_TOKEN_R'])
streamer = TextStreamer(tokenizer, skip_prompt=True)

# creating a pipeline for the model
pipe = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    trust_remote_code=True,
    device_map="auto",
    max_new_tokens=512, # maximum new tokens to be generated
    temperature=0.1, # increase or decrease creativity
    repetition_penalty=1.1, # output begins repeating itself
    streamer=streamer
    
    
)

qa_pipe = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    trust_remote_code=True,
    device_map="auto",
    max_new_tokens=25, # maximum new tokens to be generated
    temperature=0.1, # increase or decrease creativity
    repetition_penalty=1.1, # output begins repeating itself
    
    
)





db = setup_vector_search(os.path.join('metadata.json'), tokens_per_chunk=1024, chunk_overlap=50)

local_llm = HuggingFacePipeline(pipeline=pipe)

qa_llm = HuggingFacePipeline(pipeline=qa_pipe)

 
    
answer_template = """<s>[INST]
You are a helpful assistant named BOT that helps people find which andrew huberman podcasts to listen to.
Here is the context for the question:\n\n
{CONTEXT}

Here is what the user said:\n
User: {INPUT}

BOT:[/INST]"""


answer_prompt = PromptTemplate(template=answer_template, input_variables=['INPUT', 'CONTEXT'])
answer_chain = LLMChain(llm=local_llm, prompt=answer_prompt)



2024-02-09 11:41:30.290676: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-09 11:41:30.686124: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-09 11:41:30.686334: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-09 11:41:30.754338: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-09 11:41:30.896950: I tensorflow/core/platform/cpu_feature_guar

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/paganinik/.cache/huggingface/token
Login successful
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/paganinik/.cache/huggingface/token
Login successful


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

cwd is:  /home/paganinik/huberman_search
Loading...
Splitting...


/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Embedding...


# Where filtering
https://github.com/chroma-core/chroma/blob/main/examples/basic_functionality/where_filtering.ipynb

In [7]:
from datetime import datetime



huggingface_embedding_name="BAAI/bge-small-en-v1.5"


embedding_function = HuggingFaceEmbeddings(model_name=huggingface_embedding_name)





cond = 1

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)


def extract_content(doc):
    text = 'Date published' + doc.metadata['Date'] + '\n'
    text += doc.page_content
    source = (doc.metadata['title'], doc.metadata['source'], doc.metadata['Date'])
    return text, source


def find_date(date_string):
  

    # Parse the date string into a datetime object
    dt_object = datetime.strptime(date_string, '%a, %d %b %Y %H:%M:%S %z')

    # Extract year, month, and day
    year = str(dt_object.year)
    month = str(dt_object.month)
    day = str(dt_object.day)

    # Zero-pad the month and day if it's a single digit
    month = month.zfill(2)
    day = day.zfill(2)
    
        
    return year + '-' + month + '-' + day


def find_file(date_string):
    files = os.listdir('huberman-podcasts')
    for f in files:
        if date_string in f:
            return f

        
def load_db(f):
    if f is not None:
        print(f'Trying to load {f}')
        with open(os.path.join('huberman-podcasts', f)) as working_file:
            text = working_file.read()
            chunks = text_splitter.create_documents([text])
            d = Chroma.from_documents(chunks, embedding_function)
            return d


while cond == 1:

    
    
    user_query = input("User: ")
    context = db.similarity_search(user_query, k=5)
    text = []
    source = []
    for doc in context:
        t, s = extract_content(doc)
        text.append(t)
        source.append(s)
    print(f'Here are the sources:')
    [print(f'{s[0]} | {s[1]} | {s[2]}') for s in source]
    full_description_context = "\n".join(text)
    
    dates = list(set([s[2] for s in source]))
    
    print(dates)
    dbs = [load_db(find_file(find_date(s))) for s in dates]
    
    context_transcript = []
    for d in dbs:
        if d is not None:
            print(d)
            context_transcript.extend([x.page_content for x in d.similarity_search(user_query, k=3)])
            
    # free memory immediately      
    del dbs
    gc.collect()
    
    context_transcript = "\n\n\n".join(context_transcript)
    
    print(context_transcript, '\n\nHUBERBOT: ')
    response = answer_chain.run({
        'INPUT': user_query,

        'CONTEXT': context_transcript
    })
    
    
    

User: What are some ways I can increase my sex drive
Here are the sources:
Dr. Rena Malik: Improving Sexual & Urological Health in Males and Females | https://hubermanlab.com/dr-rena-improving-sexual-and-urological-health-in-males-and-females | Mon, 14 Aug 2023 08:00:00 -0000
['Mon, 12 Dec 2022 09:00:00 -0000', 'Mon, 06 Nov 2023 09:00:00 -0000', 'Mon, 14 Aug 2023 08:00:00 -0000', 'Mon, 12 Apr 2021 08:00:00 -0000']
Trying to load 2022-12-12-dr-kyle-gillett-tools-for-hormone-optimization-in-males-huberman-lab-podcast-102.txt
Trying to load 2023-11-06-dr-michael-eisenberg-improving-male-sexual-health-function-fertility.txt
Trying to load 2023-08-14-dr-rena-malik-improving-sexual-urological-health-in-males-and-females-huberman-lab-podcast.txt


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


How to Optimize Testosterone & Estrogen


How To Increase Motivation & Drive
This episode explains the science of motivation and drive. I describe how dopamine-- a chemical we all make in our brain, underlies our desire for and pursuit of our goals, as well as our capacity to move and experience pleasure. I describe how we can leverage specific behaviors, reward schedules and dopamine-prolactin balance to help ensure we can maintain motivation and capacity for pleasure over the long term. I also discuss dopamine in the context of ADHD, craving and addiction, and some absolutely amazing results about specificity of drug effects based purely on belief.
For the full show notes, visit hubermanlab.com.
How To Increase Motivation & Drive


hormonal, and muscular, and connective tissue growth is key provided it's not overtraining, but so is casual exercise like walking, and stretching, and the sorts of things that will then return blood flow to that area. Is that an overly basic way to think 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In this episode, I explain how nicotine impacts the brain and body, including its potent ability to enhance attention, focus, and alertness, increase blood pressure and metabolism and reduce appetite. I discuss nicotine’s ability to increase the action of neurochemicals, including dopamine, norepinephrine, and acetylcholine and activate sympathetic (alertness-promoting) neural circuits. I also discuss common nicotine delivery methods, such as cigarettes, vaping, dip, and snuff, and how they each create their own unique experience and how they, but not nicotine itself, cause cancer and other adverse health effects. I also explain science-based tools to permanently quit smoking cigarettes or vaping, including peer-reviewed clinical hypnosis tools, antidepressants, and alternative nicotine replacement (patches, lozenges, gums etc.). As nicotine is one of the most widely used substances with billions of users — most of whom report wanting to quit — this episode ought to be of interest to



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The Effects of Cannabis (Marijuana) on the Brain & Body


In this episode, I discuss cannabis (aka marijuana), including the biological mechanisms underlying its effects on the mind and body, its known medical applications, its impact on libido, creativity, hunger, hormones and more. I also cover the known adverse health consequences of chronic and even acute (one-time) use and the factors that determine if cannabis is helpful or harmful. Additionally, I detail how the various strains of cannabis: sativa, indica and hybrid strains, can produce such divergent effects depending on the strain type, THC-to-CBD ratio, total dosage, and frequency of use. I review why cannabis can impact speech patterns and one’s propensity to develop anxiety/depression during and after use and, in some individuals, paranoia. As the legal landscape for cannabis is rapidly evolving, this episode should interest a wide audience, including former/current cannabis users, those in the medical, sports, law enforcem

KeyboardInterrupt: Interrupted by user